In [2]:
from typing import Any , Dict
from google.adk.agents import Agent , LlmAgent
from google.adk.apps.app import App , EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService, InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types

In [43]:
#Agent Executor Function

async def agent_invoke(
        runner: Runner,
        user_input: list[str] | str = None,
        session_name :str = "default_session",
):
    
   
    app_name = runner.app_name

    session = await session_service.get_session(app_name = app_name, session_id = session_name, user_id = USER_ID)

    if session:
        print(f"Resuming session: {session_name}")
    else:
        session = await session_service.create_session(app_name = app_name, session_id = session_name, user_id = USER_ID)
        print(f"creating new session: {session_name}")


    if user_input:
        if type(user_input) == str:
            user_input = [user_input]

        for msg in user_input:
            print(f"user: {msg}")

            msg = types.Content(role="user", parts=[types.Part(text = msg)])    

            async for event in runner.run_async(user_id = USER_ID , session_id = session.id , new_message = msg):
                if event.content and event.content.parts:
                    text = event.content.parts[0].text
                    print(f"model: {text}")
    else:
        print(f"No querries found.")
    

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

In [5]:

root_agent = LlmAgent(
    name= "Chatbot_Agent",
    model= Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="A memory based Chatbot."
)

print("Agent Created.")

Agent Created.


In [6]:
APP_NAME = "agentic_ai_03"
USER_ID = "default"
SESSION = "default"

session_service = InMemorySessionService()

runner = Runner(
    agent= root_agent,
    app_name= APP_NAME,
    session_service= session_service
)

App name mismatch detected. The runner is configured with app name "agentic_ai_03", but the root agent was loaded from "D:\Google Course\myenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [7]:
result = await agent_invoke(
    runner,
    ["Hello my name is Aamish.? I want to learn Machine learning.",
     "can you help me for building a roadmap.",
     "what is my name?"],

    "Aamish_Session"
)

creating new session: Aamish_Session
user: Hello my name is Aamish.? I want to learn Machine learning.
model: Hello Aamish! It's great to meet you. Machine learning is a fascinating field, and I'm here to help you learn it.

Since you're just starting, we can begin with the fundamentals. How about we start with:

1.  **What is Machine Learning?** We can define it, discuss its importance, and look at some real-world examples.
2.  **Types of Machine Learning:** Understanding the different categories like Supervised, Unsupervised, and Reinforcement Learning.
3.  **Basic Concepts:** Introducing terms like data, features, models, training, and prediction.

Does this sound like a good starting point for you, Aamish? We can adjust as we go based on what you find interesting or challenging.
user: can you help me for building a roadmap.
model: Absolutely, Aamish! Building a roadmap is a fantastic idea to structure your learning. Here’s a suggested roadmap for learning Machine Learning, broken d

Now we Use DBMS to save our Memory

In [8]:
APP_NAME = "agents"
USER_ID = "default"
SESSION = "default"
db_url = "sqlite:///Agentic_AI_03.db"

session_service = DatabaseSessionService(db_url= db_url)

runner = Runner(
    agent= root_agent,
    app_name= APP_NAME,
    session_service= session_service
)

print("Agent with DBMS Created.")

Agent with DBMS Created.


In [44]:
result = await agent_invoke(
    runner,
    ["Hello my name is Aamish.? I want to learn Machine learning.",
     "can you help me for building a roadmap.",
     "what is my name?"],

    "Aamish_session_DB_02"
)

creating new session: Aamish_session_DB_02
user: Hello my name is Aamish.? I want to learn Machine learning.


model: Nice to meet you, Aamish! Machine learning is a fascinating field. While I can't directly teach you, I can help you find resources or explain concepts once you have specific questions.

To get started, would you like me to save your name? Also, could you tell me which country you're in? This might help me tailor any resource suggestions later.
model: None
user: can you help me for building a roadmap.
user: can you help me for building a roadmap.
model: Of course! To build a roadmap for learning Machine Learning, let's first establish your current understanding and goals.

Could you tell me a bit about your background? For example:

*   **What is your current level of programming experience?** (e.g., beginner, intermediate, advanced in Python, R, etc.)
*   **Do you have a background in mathematics?** (e.g., linear algebra, calculus, probability, statistics)
*   **What are your specific interests within Machine Learning?** (e.g., computer vision, natural language processing, reinf

Use Of Tool Context

In [28]:
def save_userinfo(tool_context: ToolContext, user_name: str, country: str)-> dict[str,Any]:
    """Tool to save user information in which username and courntry are stored in tool context storage."""

    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


def retrive_userinfo(tool_context: ToolContext)-> dict[str,Any]:
    """Tools to retrive user information from tool context storage."""
    user_name = tool_context.state.get("user:name", "user_name_not_found")
    country = tool_context.state.get("user:country","country_not_found")
    return{"status": "success", "user_name": user_name, "country": country}

In [29]:
root_agent = LlmAgent(
    name= "text_chatbot_agent",
    model= Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description = "A text chatbot that use tool which is provided for storing and retriving user information.",
    tools=[ save_userinfo, retrive_userinfo]
)

session_service = InMemorySessionService()

runner = Runner(
    agent = root_agent,
    session_service= session_service,
    app_name = 'agents'
)

print("Agent with Tool Context Created.")

Agent with Tool Context Created.


In [45]:
result = await agent_invoke(
    runner,
    ["Hello my name is Aamish and I am from Pakistan."
    "can you retrive my information?"],
    "tool_context_sesssion"
)

Resuming session: tool_context_sesssion
user: Hello my name is Aamish and I am from Pakistan.can you retrive my information?


model: None
model: None
model: I have saved your information and I can retrieve it. You are Aamish and you are from Pakistan.
model: I have saved your information and I can retrieve it. You are Aamish and you are from Pakistan.
